In [13]:
import numpy as np
import random
from time import time

In [14]:
class Server:
  def __init__(self):
    pass
  
  def process_guess(solution, guess):
    '''
    Note: I have been trying to perform this calculation using multiplication, but
    the only way I can think of is to do a multiplication in mod26 and check the
    remainder to see if the guesses are in the correct spot. However, I do not
    know how to perform the modulus multiplication (or if it is even supported)
    in the current SEAL library.
    '''
    print('\tComputations starting here are performed on the SERVER.')
    print('\tServer view of solution: {}'.format(solution))
    print('\tServer view of guess: {}'.format(guess))
    result1 = solution - guess[0]
    result2 = solution - guess[1]
    result3 = solution - guess[2]
    result4 = solution - guess[3]
    result5 = solution - guess[4]
    t_end = time()
    print('Computations starting here are NO LONGER performed on the SERVER.')
    return (result1, result2, result3, result4, result5)

class Client:
  def __init__(self, words=['hello','world']):    
    self.word_list = words
  
  def split(self, guess):
    g1 = np.array([guess[0]] * 5)
    g2 = np.array([guess[1]] * 5)
    g3 = np.array([guess[2]] * 5)
    g4 = np.array([guess[3]] * 5)
    g5 = np.array([guess[4]] * 5)
    return [g1, g2, g3, g4, g5]

  def process_and_decrypt(self, encrypted):
    return [encrypted[0], encrypted[1], encrypted[2], encrypted[3], encrypted[4]]

  def play(self):
    while True:
      #play game loop
      print(' x  represents that "x" is in the CORRECT spot.')
      print('[x] represents that "x" is in the word, but NOT the correct spot.')
      print('(x) represents that "x" is NOT in the word.')
      print('Please only guess 5-letter words!')
      print('Start game? (y/n):')
      resp = input()
      resp = resp.lower()
      if resp == 'n':
        print('Thanks for playing!')
        break
      elif resp == 'y':
        print('Computations starting here are performed on the CLIENT.')
        #the solution word and guesses are assumed to be perform from the client
        #calculations are assumed to be performed on the side of the server
        solution = random.choice(self.word_list)
        solution_ascii = [] #get int-value of the word, the first plaintext vector
        for c in solution:
          solution_ascii.append(ord(c))
        solution_ascii = np.array(solution_ascii)
                
        #start guessing loop
        while True:
          print('Please enter your guess: ', end='')
          guess = input()
          guess = guess.lower()
          guess_ascii = []
          for c in guess: #get int-value of the guess, the second plaintext vector
            guess_ascii.append(ord(c))
          #Start timing of encrypted computations
          t_start = time()

          #mimic sending to server and process
          response = Server.process_guess(solution_ascii, self.split(guess_ascii))
          print('Client view of response: {}'.format(response))
          #End timing of encrypted computations
          t_end = time()
          print('\tTotal computations took: {} ms'.format((t_end-t_start)*1000))
          output = ''
          total = 0
          for i in range(5):
            inner_total = 0
            if response[i][i] == 0: #if the character is in the right spot
              output += '{} '.format(guess[i])
              total += 1
            else:
              for j in range(5):
                if response[i][j] == 0: #check remaining spots
                  if inner_total == 0:
                    output += '[{}]'.format(guess[i])
                  inner_total += 1
              if inner_total == 0:
                output += '({})'.format(guess[i])
          print('Response from server: {}'.format(output))
          if total >= 5:
            print('CONGRATULATIONS! You guessed the word!')
            break
      else:
        print('Invalid input, please enter "y" or "n"!')

if __name__ == '__main__':
  S = Server()
  C = Client()
  C.play()

 x  represents that "x" is in the CORRECT spot.
[x] represents that "x" is in the word, but NOT the correct spot.
(x) represents that "x" is NOT in the word.
Please only guess 5-letter words!
Start game? (y/n):
y
Computations starting here are performed on the CLIENT.
Please enter your guess: start
	Computations starting here are performed on the SERVER.
	Server view of solution: [119 111 114 108 100]
	Server view of guess: [array([115, 115, 115, 115, 115]), array([116, 116, 116, 116, 116]), array([97, 97, 97, 97, 97]), array([114, 114, 114, 114, 114]), array([116, 116, 116, 116, 116])]
Computations starting here are NO LONGER performed on the SERVER.
Client view of response: (array([  4,  -4,  -1,  -7, -15]), array([  3,  -5,  -2,  -8, -16]), array([22, 14, 17, 11,  3]), array([  5,  -3,   0,  -6, -14]), array([  3,  -5,  -2,  -8, -16]))
	Total computations took: 9.006023406982422 ms
Response from server: (s)(t)(a)[r](t)
Please enter your guess: world
	Computations starting here are p